In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import csv
import xlrd
import io
from zipfile import ZipFile
from datetime import date, timedelta, time, datetime

In [2]:
today_yyyymmdd = date.today().strftime("%Y%m%d")
print(today_yyyymmdd)

20200714


import datetime 
base = datetime.date(2020,2,1)
for i in range(0,10): 
    d=base + datetime.timedelta(i)
    d=d.strftime("%Y%m%d")
    print(f"N029kakutokuruiseki{d}.csv")

from datetime import datetime, timedelta

def date_range(start_date: datetime, end_date: datetime):
    diff = (end_date - start_date).days + 1
    return (start_date + timedelta(i) for i in range(diff))

In [3]:
#pandasの表示列数を増やす
import pandas as pd
pd.set_option('display.max_columns', 100)

In [4]:
data_dir = Path(r"C:\Users\crie072\Documents\21_Online")

#新規入会リスト from CSC
input_file = "【C100】新規入会者_5月末時点.xlsx"
output_file = f"集計_新規入会者_5月末時点_{today_yyyymmdd}"

#累積アフィリエイト承認リスト
input_file_2 = "アフィリエイト広告_累積承認者リスト.xlsx"

act_month = 202005

#アフィリエイト広告別実績
output_file_2 = f"アフィリエイト広告別実績_5月末時点_{today_yyyymmdd}"
print(input_file)
print(output_file)
print(input_file_2)
print(output_file_2)

【C100】新規入会者_5月末時点.xlsx
集計_新規入会者_5月末時点_20200714
アフィリエイト広告_累積承認者リスト.xlsx
アフィリエイト広告別実績_4月末時点_20200714


column_names = ['実績月','入会申込番号','直近３ケ月買物利用金額','直近３ケ月全体利用金額','年令','性別',
                '都道府県名','顧客状況コ－ド','マイペ設定','DM希望店','カード券種名','カ－ド加入日','初回利用日',
                'あとからリボ申込有無コード','Weｂ明細有無コード','解約','入会チャネル']

In [17]:
df = pd.read_excel(data_dir /r"input"/f"{input_file}")
df.head()

,実績月,入会申込番号,直近６ケ月買物利用金額,直近６ケ月全体利用金額,年令,性別,都道府県名,顧客状況コ－ド,マイペ設定,DM希望店コード,カード券種名,カ－ド加入日,初回利用日,あとからリボ申込有無コード,Weｂ明細有無コード,解約,入会チャネル
0,202005,155051921627812,769,769,38,女性,東京都,0,0,M_上野店,大丸松坂屋VISAゴールド,20200106,20200116,0,1,0,オンライン
1,202005,155051921628414,600,600,27,男性,兵庫県,0,0,Ｄ_神戸店,大丸松坂屋VISAゴールド,20200106,20200118,0,1,0,オンライン
2,202005,155051921636805,143,143,63,男性,大阪府,0,0,Ｄ_梅田店,大丸松坂屋VISAゴールド,20200106,20200115,0,1,0,オンライン
3,202005,155193017345014,2,2,46,男性,茨城県,0,1,Ｄ_東京店,大丸松坂屋VISAゴールド,20200106,20200116,0,1,0,店頭タブレット
4,202005,155193017345790,148,148,59,男性,兵庫県,0,1,Ｄ_梅田店,大丸松坂屋VISAゴールド,20200106,20200209,0,1,0,店頭タブレット


In [18]:
print(df.shape)
print(df.index)
print(df.columns)
print(df.dtypes)

(31469, 17)
RangeIndex(start=0, stop=31469, step=1)
Index(['実績月', '入会申込番号', '直近６ケ月買物利用金額', '直近６ケ月全体利用金額', '年令', '性別', '都道府県名',
       '顧客状況コ－ド', 'マイペ設定', 'DM希望店コード', 'カード券種名', 'カ－ド加入日', '初回利用日',
       'あとからリボ申込有無コード', 'Weｂ明細有無コード', '解約', '入会チャネル'],
      dtype='object')
実績月               int64
入会申込番号            int64
直近６ケ月買物利用金額       int64
直近６ケ月全体利用金額       int64
年令                int64
性別               object
都道府県名            object
顧客状況コ－ド          object
マイペ設定             int64
DM希望店コード         object
カード券種名           object
カ－ド加入日            int64
初回利用日             int64
あとからリボ申込有無コード     int64
Weｂ明細有無コード        int64
解約                int64
入会チャネル           object
dtype: object


In [19]:
df['カ－ド加入日'] = pd.to_datetime(df['カ－ド加入日'], format = '%Y%m%d')
df['カ－ド加入年'] = df['カ－ド加入日'].dt.year
df['カ－ド加入月'] = df['カ－ド加入日'].dt.month
df.head()

,実績月,入会申込番号,直近６ケ月買物利用金額,直近６ケ月全体利用金額,年令,性別,都道府県名,顧客状況コ－ド,マイペ設定,DM希望店コード,カード券種名,カ－ド加入日,初回利用日,あとからリボ申込有無コード,Weｂ明細有無コード,解約,入会チャネル,カ－ド加入年,カ－ド加入月
0,202005,155051921627812,769,769,38,女性,東京都,0,0,M_上野店,大丸松坂屋VISAゴールド,2020-01-06,20200116,0,1,0,オンライン,2020,1
1,202005,155051921628414,600,600,27,男性,兵庫県,0,0,Ｄ_神戸店,大丸松坂屋VISAゴールド,2020-01-06,20200118,0,1,0,オンライン,2020,1
2,202005,155051921636805,143,143,63,男性,大阪府,0,0,Ｄ_梅田店,大丸松坂屋VISAゴールド,2020-01-06,20200115,0,1,0,オンライン,2020,1
3,202005,155193017345014,2,2,46,男性,茨城県,0,1,Ｄ_東京店,大丸松坂屋VISAゴールド,2020-01-06,20200116,0,1,0,店頭タブレット,2020,1
4,202005,155193017345790,148,148,59,男性,兵庫県,0,1,Ｄ_梅田店,大丸松坂屋VISAゴールド,2020-01-06,20200209,0,1,0,店頭タブレット,2020,1


In [20]:
df['入会チャネル'].unique()

array(['オンライン', '店頭タブレット', '郵送'], dtype=object)

df[df['入会チャネル']=='オンライン'].head()

In [21]:
df['実績月'].unique()

array([202005], dtype=int64)

df = df[df['実績月'] == 202003]
print(df.shape)

In [22]:
#入会申込番号の重複がないかチェック
print(len(df) == len(df['入会申込番号'].unique()))

False


In [23]:
#重複削除
df_dup = df.drop_duplicates(subset=['入会申込番号','実績月'])

In [24]:
df_dup.shape[0]

26479

In [25]:
#欠損データがないかチェック
df_dup.isnull().sum()

実績月              0
入会申込番号           0
直近６ケ月買物利用金額      0
直近６ケ月全体利用金額      0
年令               0
性別               0
都道府県名            0
顧客状況コ－ド          0
マイペ設定            0
DM希望店コード         0
カード券種名           0
カ－ド加入日           0
初回利用日            0
あとからリボ申込有無コード    0
Weｂ明細有無コード       0
解約               0
入会チャネル           0
カ－ド加入年           0
カ－ド加入月           0
dtype: int64

In [26]:
#SettingWithCopyWarning回避対策
df_dup2 = df_dup.drop(columns=['あとからリボ申込有無コード'])
df_dup2['初回利用日フラグ'] = df_dup2['初回利用日'].apply(lambda x : "1" if x > 0 else "0")

In [27]:
#利用無し客を置き換え
df_dup2.loc[df_dup2['初回利用日'] <= 0, '初回利用日'] = 20301231
df_dup2.head()

,実績月,入会申込番号,直近６ケ月買物利用金額,直近６ケ月全体利用金額,年令,性別,都道府県名,顧客状況コ－ド,マイペ設定,DM希望店コード,カード券種名,カ－ド加入日,初回利用日,Weｂ明細有無コード,解約,入会チャネル,カ－ド加入年,カ－ド加入月,初回利用日フラグ
0,202005,155051921627812,769,769,38,女性,東京都,0,0,M_上野店,大丸松坂屋VISAゴールド,2020-01-06,20200116,1,0,オンライン,2020,1,1
1,202005,155051921628414,600,600,27,男性,兵庫県,0,0,Ｄ_神戸店,大丸松坂屋VISAゴールド,2020-01-06,20200118,1,0,オンライン,2020,1,1
2,202005,155051921636805,143,143,63,男性,大阪府,0,0,Ｄ_梅田店,大丸松坂屋VISAゴールド,2020-01-06,20200115,1,0,オンライン,2020,1,1
3,202005,155193017345014,2,2,46,男性,茨城県,0,1,Ｄ_東京店,大丸松坂屋VISAゴールド,2020-01-06,20200116,1,0,店頭タブレット,2020,1,1
4,202005,155193017345790,148,148,59,男性,兵庫県,0,1,Ｄ_梅田店,大丸松坂屋VISAゴールド,2020-01-06,20200209,1,0,店頭タブレット,2020,1,1


#df_dup[['カ－ド加入日','初回利用日']] = df_dup[['カ－ド加入日','初回利用日']].applymap(str).applymap(lambda s: "{}/{}/{}".format(s[0:4],s[4:6],s[6:]))

#df_dup['カ－ド加入日'] = pd.to_datetime(df_dup['カ－ド加入日'], format='%Y-%m-%d')

In [28]:
df_dup2['初回利用日'] = pd.to_datetime(df_dup2['初回利用日'], format = '%Y%m%d').copy()
df_dup2.head()

,実績月,入会申込番号,直近６ケ月買物利用金額,直近６ケ月全体利用金額,年令,性別,都道府県名,顧客状況コ－ド,マイペ設定,DM希望店コード,カード券種名,カ－ド加入日,初回利用日,Weｂ明細有無コード,解約,入会チャネル,カ－ド加入年,カ－ド加入月,初回利用日フラグ
0,202005,155051921627812,769,769,38,女性,東京都,0,0,M_上野店,大丸松坂屋VISAゴールド,2020-01-06,2020-01-16,1,0,オンライン,2020,1,1
1,202005,155051921628414,600,600,27,男性,兵庫県,0,0,Ｄ_神戸店,大丸松坂屋VISAゴールド,2020-01-06,2020-01-18,1,0,オンライン,2020,1,1
2,202005,155051921636805,143,143,63,男性,大阪府,0,0,Ｄ_梅田店,大丸松坂屋VISAゴールド,2020-01-06,2020-01-15,1,0,オンライン,2020,1,1
3,202005,155193017345014,2,2,46,男性,茨城県,0,1,Ｄ_東京店,大丸松坂屋VISAゴールド,2020-01-06,2020-01-16,1,0,店頭タブレット,2020,1,1
4,202005,155193017345790,148,148,59,男性,兵庫県,0,1,Ｄ_梅田店,大丸松坂屋VISAゴールド,2020-01-06,2020-02-09,1,0,店頭タブレット,2020,1,1


df_dup['実績月'] = df_dup['実績月'] * 100 + 1

df_dup['実績月'] = pd.to_datetime(df_dup['実績月'], format = '%Y%m%d')

df_dup[['実績月2']] = df_dup[['実績月']].applymap(str).applymap(lambda s: "{}/{}/{}".format(s[0:4],s[4:6],s[6:]))

df_dup['実績月2'] = pd.to_datetime(df_dup['実績月2'], format='%Y-%m-%d')

df_dup = df_dup.drop(columns=['実績月'])

集計

In [29]:
df_dup2['no'] = 1

In [30]:
#客数
df_count = df_dup2.groupby(['実績月','入会チャネル','カ－ド加入年','カ－ド加入月','初回利用日フラグ','解約'])['no'].count()

In [31]:
#金額
df_amount= df_dup2.groupby(['実績月','入会チャネル','カ－ド加入年','カ－ド加入月','初回利用日フラグ','解約'])['直近６ケ月全体利用金額'].sum()

In [32]:
df_merge = pd.merge(df_count, df_amount, on=['実績月','入会チャネル','カ－ド加入年','カ－ド加入月','初回利用日フラグ','解約'], how='inner')
df_merge.head()

no  直近６ケ月全体利用金額
実績月    入会チャネル カ－ド加入年 カ－ド加入月 初回利用日フラグ 解約                  
202005 オンライン  2020   1      0        0   380            4
                                     1    67            0
                            1        0   293        30564
                                     1    13           59
                     2      0        0   648            4

In [33]:
df_merge.to_csv(data_dir /r"output"/f"{output_file}.csv", header=True,encoding='utf-8-sig')

In [34]:
#アフィリエイト先別に実績集計

In [35]:
df_cum = pd.read_excel(data_dir/"input"/f"{input_file_2}", dtype={'参照用': str})
df_cum = df_cum.rename(columns={"参照用":'入会申込番号'})
df_cum.head()

,ASID,メディア名,媒体チャネル,クリック,申込,入会申込番号,ASP名,単価,可否,承認作業月
0,37,げん玉,ポイント,2020-01-08 19:11:08,2020-01-08 19:32:05,155051921643397,BGT,9000,可,1月
1,52,ECナビSP,ポイント,2020-01-08 20:21:44,2020-01-08 20:31:05,155051921643553,BGT,9000,可,1月
2,52,ECナビSP,ポイント,2020-01-08 20:31:50,2020-01-08 20:39:06,155051921643587,BGT,9000,可,1月
3,52,ECナビSP,ポイント,2020-01-08 20:34:03,2020-01-08 20:45:05,155051921643678,BGT,9000,可,1月
4,82,ちょびリッチ。ドットコム,ポイント,2020-01-08 20:32:12,2020-01-08 20:55:06,155051921643744,BGT,9000,可,1月


In [36]:
for i in range(len(df_cum)):
    if pd.isnull(df_cum['ASID'][i]) == True:
        df_cum.loc[:,'ASID'][i] = 0

df_cum = df_cum.astype({'ASID':int})

In [37]:
df_dup2.head()

,実績月,入会申込番号,直近６ケ月買物利用金額,直近６ケ月全体利用金額,年令,性別,都道府県名,顧客状況コ－ド,マイペ設定,DM希望店コード,カード券種名,カ－ド加入日,初回利用日,Weｂ明細有無コード,解約,入会チャネル,カ－ド加入年,カ－ド加入月,初回利用日フラグ,no
0,202005,155051921627812,769,769,38,女性,東京都,0,0,M_上野店,大丸松坂屋VISAゴールド,2020-01-06,2020-01-16,1,0,オンライン,2020,1,1,1
1,202005,155051921628414,600,600,27,男性,兵庫県,0,0,Ｄ_神戸店,大丸松坂屋VISAゴールド,2020-01-06,2020-01-18,1,0,オンライン,2020,1,1,1
2,202005,155051921636805,143,143,63,男性,大阪府,0,0,Ｄ_梅田店,大丸松坂屋VISAゴールド,2020-01-06,2020-01-15,1,0,オンライン,2020,1,1,1
3,202005,155193017345014,2,2,46,男性,茨城県,0,1,Ｄ_東京店,大丸松坂屋VISAゴールド,2020-01-06,2020-01-16,1,0,店頭タブレット,2020,1,1,1
4,202005,155193017345790,148,148,59,男性,兵庫県,0,1,Ｄ_梅田店,大丸松坂屋VISAゴールド,2020-01-06,2020-02-09,1,0,店頭タブレット,2020,1,1,1


In [38]:
#新規入会リストからオンライン客に限定
df_online =df_dup2[(df_dup2["入会チャネル"] == "オンライン") 
               & (df_dup2["実績月"] == act_month) ]
df_online = df_online.astype({'入会申込番号':str})

In [39]:
df_online.shape[0]

2946

In [40]:
df_merge = pd.merge(df_cum, df_online, on=['入会申込番号'], how='inner')
df_merge.shape[0]

1867

In [41]:
#客数
df_count = df_merge.groupby(['メディア名','実績月','入会チャネル','カ－ド加入年','カ－ド加入月','初回利用日フラグ','解約'])['no'].count()

In [42]:
#金額
df_amount= df_merge.groupby(['メディア名','実績月','入会チャネル','カ－ド加入年','カ－ド加入月','初回利用日フラグ','解約'])['直近６ケ月全体利用金額'].sum()

In [43]:
df_merge = pd.merge(df_count, df_amount, on=['メディア名','実績月','入会チャネル','カ－ド加入年','カ－ド加入月','初回利用日フラグ','解約'], how='inner')
df_merge.head()

no  直近６ケ月全体利用金額
メディア名            実績月    入会チャネル カ－ド加入年 カ－ド加入月 初回利用日フラグ 解約                 
CLUBPanasonicコイン 202005 オンライン  2020   1      1        0    4          120
                                      2      0        0    3            0
                                             1        0    1            2
                                      3      0        0    1            0
                                             1        0    1            0

In [44]:
df_merge.to_csv(data_dir /r"output"/f"{output_file_2}.csv", header=True,encoding='utf-8-sig')